# Experiment Notebook: Code Walkthrough

# Table of Contents
* [Experiment Summary](#Experiment-Summary)
* [Experiment Assumptions](#Experiment-Assumptions)
* [Experiment Setup](#Experiment-Setup)
* [Analysis 1: ...](#Analysis-1:-...)

# Experiment Summary 

The purpose of this notebook is to...

# Experiment Assumptions

See [assumptions document](../../ASSUMPTIONS.md) for further details.

# Experiment Setup

We begin with several experiment-notebook-level preparatory setup operations:

* Import relevant dependencies
* Import relevant experiment templates
* Create copies of experiments
* Configure and customize experiments 

Analysis-specific setup operations are handled in their respective notebook sections.

In [ ]:
# Import the setup module:
# * sets up the Python path
# * runs shared notebook configuration methods, such as loading IPython modules
import setup

import copy
import logging
import numpy as np
import pandas as pd
import plotly.express as px

import experiments.notebooks.visualizations as visualizations
from experiments.run import run
from experiments.utils import display_code

In [ ]:
import plotly.io as pio
png_renderer = pio.renderers["png"]
png_renderer.width = 1200
png_renderer.height = 500
# png_renderer.scale = 1

pio.renderers.default = "png"

In [ ]:
# Enable/disable logging
logger = logging.getLogger()
logger.disabled = False

In [ ]:
# Import experiment templates
import experiments.default_experiment as default_experiment

In [ ]:
# Inspect experiment template
display_code(default_experiment)

In [ ]:
# Create a simulation for each analysis
simulation_1 = copy.deepcopy(default_experiment.experiment.simulations[0])

In [ ]:
# Experiment configuration
simulation_1.runs = 5

simulation_1.model.initial_state.update({})

simulation_1.model.params.update({
    "rebalancing_period": [30, 60, 90]
})

In [ ]:
# TODO: where can this be put? Utils?
def plotly_to_static(plot, width=900, height=500, scale=1):
    return Image(plot.to_image(format="png", width=width, height=height, scale=scale))

# Analysis 1: Model Output

A visualization of model outputs based on existing implementation of 
- PCV Management and Rebalancing
- Liquidity Pool CFMM Dynamics

In [ ]:
# Analysis-specific setup

In [ ]:
# Experiment execution
df, exceptions = run(simulation_1)

In [ ]:
# Post-processing and visualizations

In [ ]:
df

## Asset Prices

Asset prices are implemented as two types of stochatic processes:
- Volatile Assets: Brownian Motion $$dX_t = \mu_t dt + \sigma_t dW_t, \ W_t \sim BM(0,1)$$
- Stable Assets: Gaussian Noise $$X_t = \mu + \sigma Z_t, Z_t \sim N(0,1)$$

Volatile Asset price process for monte carlo runs for parameter values: $$\mu=-50, \sigma=20$$

In [ ]:
Stable Asset price process for monte carlo runs for parameter values: $$\mu=1, \sigma=0.005$$

Stable Asset price process for monte carlo runs for parameter values: $$\mu=1, \sigma=0.005$$

In [ ]:
df.query("subset == 0").plot(x='timestep', y='stable_asset_price', color='run')

A realization of a single stable asset process run, compared to FEI's current implementation as equal to $1 by construction

In [ ]:
fig = df.query("run == 1 and subset == 0").plot(x='timestep', y=['stable_asset_price', 'fei_price'])
plotly_to_static(fig)

A realization of a single stable asset process run, compared to FEI's current implementation as equal to $1 by construction

## Protocol Controlled Value

The main <b>monetary policy lever</b> implemented and defining the dynamics of this block is <b>PCV rebalancing</b>. The current implementation is a naive <i>stable-backing-ratio based</i> rebalancing strategy. Where if the stable backing ratio is below an arbitrary value (50%) in this case, the necessary amount of volatile assets is sold and the corresponding amount of stable assets is bought. The test for stable backing ratio comparison occurs arbitrarily for three parameter values of rebalancing frequency:
- 30 days
- 60 days
- 90 days

The strategy pays no attention to dollar cost averaging the entry/exit of respective positions nor does it take any other parameters into account, as such, the dynamics are largely impulsive.

The stable backing ratio resulting from the rebalancing policy for various rebalancing frequency dates. The stable backing ratio for the current run never falls below 50% after 200 timesteps.

In [ ]:
fig = df.query('run == 1').plot(
    x='timestep',
    y=['stable_backing_ratio'],
    title='Stable Backing Ratio',
    color='subset'
)

fig.update_xaxes(title='Timestep (days)')
fig.update_yaxes(title='Stable Backing Ratio')

An overall look at PCV rebalancing. At period 30,roughly 140M of Volatile asset is sold, and 140M worth of Stable asset is bought. This does not affect the total PCV value but changes the stable backing ratio to the one desired.

In [ ]:
fig = df.query("run == 1 and subset == 0").plot(
    x='timestep',
    y=['total_volatile_asset_pcv', 'total_stable_asset_pcv', 'total_pcv'],
    title='Stable and Volatile Asset PCV Totals',
)

fig.update_xaxes(title='Timestep (days)')
fig.update_yaxes(title='PCV (USD)')

plotly_to_static(fig, width=1200)

An overall look at PCV rebalancing. At period 30,roughly 140M of Volatile asset is sold, and 140M worth of Stable asset is bought. This does not affect the total PCV value but changes the stable backing ratio to the one desired.

The <b>dollar value</b> of the Stable Asset constituent of PCV across rebalancing frequencies.

In [ ]:
fig = df.query('run == 1').plot(
    x='timestep',
    y='total_stable_asset_pcv',
    title='Stable Asset Protocol Controlled Value',
    color='subset'
)

fig.update_xaxes(title='Timestep (days)')
fig.update_yaxes(title='Stable Asset PCV (USD)')

plotly_to_static(fig)

The <b>dollar value</b> of the Stable Asset constituent of PCV across rebalancing frequencies.

The <b>dollar value</b> of the Volatile Asset constituent of PCV across rebalancing frequencies.

In [ ]:
fig = df.query('run == 1').plot(
    x='timestep',
    y='total_volatile_asset_pcv',
    title='Volatile Asset Protocol Controlled Value',
    color='subset'
)

fig.update_xaxes(title='Timestep (days)')
fig.update_yaxes(title='Volatile Asset PCV (USD)')

The <b>dollar value</b> of PCV across rebalancing frequencies.

In [ ]:
fig = df.query('run == 1').plot(
    x='timestep',
    y='total_pcv',
    title='Total Protocol Controlled Value',
    color='subset',
)

fig.update_xaxes(title='Timestep (days)')
fig.update_yaxes(title='Total PCV (USD)')

plotly_to_static(fig)

The collateral ratio series over time. This is, by construction, strongly correlated with PCV Value, and in practice driven by the realization of the ETH price process for the current run.

In [ ]:
fig = df.query('run == 1').plot(
    x='timestep',
    y=['collateralization_ratio'],
    title='Collateralization Ratio',
    color='subset'
)

fig.update_xaxes(title='Timestep (days)')
fig.update_yaxes(title='Collateralization Ratio')

plotly_to_static(fig)

The collateral ratio series over time. This is, by construction, strongly correlated with PCV Value, and in practice driven by the realization of the ETH price process for the current run.

## Liquidity Pool Source / Sink

Here we look at the dynamics of <b>FEI sourced and sinked</b> as a result of CFMM movements. These dynamics do <b>not</b> tie into broader PCV management save for the basic accounting of FEI and Volatile asset amount held.

## Liquidity Pool Source / Sink

Here we look at the dynamics of <b>FEI sourced and sinked</b> as a result of CFMM movements. These dynamics do <b>not</b> tie into broader PCV management save for the basic accounting of FEI and Volatile asset amount held.

In [ ]:
def get_pcv_balance_from_deposits(df, asset, deposit_type, subset=0):
    
    col_str = asset+'_deposit_'+deposit_type
    
    df_ = df.query("subset == @subset")[[col_str, 'run', 'timestep']]
    df_[col_str+'_balance'] = df_[col_str].map(lambda x: x.balance)
    return df_[[col_str+'_balance', 'run', 'timestep']]

Liquidity pool TVL for 5 monte carlo runs for the same underlying volatile asset price parameters.

In [ ]:
df.query("subset == 0").plot(x='timestep', y ='liquidity_pool_tvl', color='run', title='FEI/VOL LP TVL',)

FEI balance, in <b>FEI units (FEI)</b>, in the liquidity pool, for 5 monte carlo runs for the same underlying volatile asset price parameters.

In [ ]:
get_pcv_balance_from_deposits(df, 'fei', 'liquidity_pool', subset=0).plot(
    title='FEI Balance of LP (in FEI)',
    x='timestep',
    y='fei_deposit_liquidity_pool_balance',
    color='run'
)

Volatile asset balance, in <b>Volatile Asset Units</b> in the liquidity pool, for 5 monte carlo runs for the same underlying volatile asset price parameters.

In [ ]:
get_pcv_balance_from_deposits(df, 'volatile', 'liquidity_pool', subset=0).plot(
    title='Volatile Asset Balance of LP (in Volatile Asset Units)',
    x='timestep',
    y='volatile_deposit_liquidity_pool_balance',
    color='run'
)

Here, we broadly illustrate the dynamics captured in the corresponding CFMM spreadhseet - As ETH price decreases, FEI amount in pool decreases and ETH amount in pool increases. Need more ETH per unit of FEI. As a result, TVL decreases.

The time series of FEI LP balance differential in subsequent time periods for a single run.

In [ ]:
fig = fei_amt.plot(
    title='FEI Released into Market (in FEI)',
    x='timestep',
    y='fei_released_into_market',
    color='run'
)
plotly_to_static(fig)

As per the spreadsheet, we proceed to calculate <b>FEI Released to Market</b> with respect to t=0, which is the current FEI LP balance minus the balance at time zero, in number of FEI tokens.

In [ ]:
fei_amt = get_pcv_balance_from_deposits(df, 'fei', 'liquidity_pool', subset=0)
fei_amt_init = fei_amt.iloc[0]['fei_deposit_liquidity_pool_balance']

fei_amt['fei_released_into_market'] = -(fei_amt['fei_deposit_liquidity_pool_balance'] - fei_amt_init)

In [ ]:
fei_amt.plot(
    title='FEI Released into Market (in FEI)',
    x='timestep',
    y='fei_released_into_market',
    color='run'
)